In [1]:
import torch
from torch import nn, optim

import torch.nn.functional as F
from einops import rearrange
from typing import Optional
from ember import (
    Transformer,
    GroupedQueryAttn,
    MultiHeadLatentAttn,
    HFTokenizer,
    TopKSampler
)

/Users/ryanpegoud/Documents/Projects/ember/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(0)

In [3]:
B, S, D = 8, 1024, 512
mla_kwargs = {
    "latent_dim": 256,
    "pos_dim": 128,
    "n_heads": 16,
}
gqa_kwargs = {
    "n_query_heads": 8,
    "n_query_groups": 4,
}
# 
# tk = Tokenizer()
tk = HFTokenizer(model = "openai-community/gpt2")
# model = Transformer(
#     vocab_size=len(tk.base),
#     model_dim=D,
#     hidden_dim=1024,
#     attn_module=GroupedQueryAttn,
#     attn_kwargs=gqa_kwargs,
#     n_attn_blocks=3,
# )

model = Transformer(
    vocab_size=tk.vocab_size,
    model_dim=D,
    hidden_dim=1024,
    attn_module=MultiHeadLatentAttn,
    attn_kwargs=mla_kwargs,
    n_attn_blocks=3,
)

<bound method Module.parameters of Transformer(
  (embed): Embedding(50258, 512)
  (attn_blocks): ModuleList(
    (0-2): 3 x AttentionBlock(
      (mlp): SwiGLU(
        (W): Linear(in_features=512, out_features=682, bias=False)
        (V): Linear(in_features=512, out_features=682, bias=False)
        (W2): Linear(in_features=682, out_features=512, bias=False)
      )
      (norm): RMSNorm()
      (attn): MultiHeadLatentAttn(
        (fused_qkv_down_proj): Linear(in_features=512, out_features=768, bias=True)
        (q_up_proj): Linear(in_features=256, out_features=512, bias=True)
        (k_up_proj): Linear(in_features=256, out_features=512, bias=True)
        (v_up_proj): Linear(in_features=256, out_features=512, bias=True)
        (q_pos_proj): Linear(in_features=256, out_features=128, bias=True)
        (k_pos_proj): Linear(in_features=512, out_features=8, bias=True)
        (o_proj): Linear(in_features=512, out_features=512, bias=True)
        (rope): RoPE()
      )
    )
  )
  (

In [4]:
txt = ["Hey there", "How are you?"]
x = tk(txt, mode="inference")
tk.decode(x)

['<|pad|><|pad|>Hey there', 'How are you?']

In [5]:
out = model.generate(
    txt,
    max_new_tokens=50,
    sampler=TopKSampler(50),
    tokenizer=tk,
)
out

['<|pad|><|pad|>Hey there BertClaim Sim moreover notor troop Tutorial worth troop Clinical———————————————— idiotegu routine PACK DOE DOE Obs marches Bott exchange Bott swirlingRush Ricanigntyigntyegu Applied rel monsters usability rel prioritiesatta Reese gasped idiotbc Assistant visionary rel wastelandgradient 113amples Bott idiottall Audiolution',
 'How are you? remind elapsedvalue Monday Kalshake Veget tagsNap asteroids dividends problems Crown Episcopal Soci specifically66 specifically Bes mould trafficking Episcopallinedocadoocado Collabor RS RS Crown":"","dra behavioraliris Ascension EU dividendsmatter Collabor upstream RS specificallyshake regulates ignoredYuph Credit159 Bes RS Collabor']